In [60]:
import requests
import xml.etree.ElementTree as ET
import difflib
import csv


def get_id_by_name(name):
    search_url = 'https://boardgamegeek.com/xmlapi/search?search=' + name + '&exact=1'
    
    x = requests.get(search_url)
    bgg_id = -1
    
    if(x.status_code == 200):
        parser = ET.XMLPullParser(['start'])
        parser.feed(x.text)

        for event, elem in parser.read_events():
            if(elem.tag == 'boardgame'):
                bgg_id = elem.get('objectid')
    
    if(bgg_id == -1):
        bgg_id = not_exact_search(name)
    
    return bgg_id

def get_weight_by_id(id):
    test_url = "https://boardgamegeek.com/xmlapi2/thing?id=" + id + "&stats=1"

    x = requests.get(test_url)
    weight = 0.0
    numweight = 0
    
    if(x.status_code == 200):      
        parser = ET.XMLPullParser(['start'])
        parser.feed(x.text)

        for event, elem in parser.read_events():
            if(elem.tag == 'numweights'):
                numweight = elem.get('value')
            if(elem.tag == 'averageweight'):
                weight = elem.get('value')
        
    return weight, numweight
        
            
#search by name without the exact attribute. Three possibilities:
# 1. Find nothing and so removing the record from the dataset
# 2. Find only 1 game and so this is the one that I have been searching for
# 3. Find more than 1 game for an entry. Choose between them the most similar
def not_exact_search(name):
    search_url = 'https://boardgamegeek.com/xmlapi/search?search=' + name

    x = requests.get(search_url)
    bgg_id = -1
    s = []
    
    if(x.status_code == 200):
        text = x.text
        
        parser = ET.XMLPullParser(['start'])
        parser.feed(text)
        
        result_count = 0
        
        
        for event, elem in parser.read_events():
            if(elem.tag == 'boardgame'):
                result_count += 1
                #bgg_id = elem.get('objectid')
        
        if(result_count == 1):
            parser = ET.XMLPullParser(['start'])
            parser.feed(text)
            for event, elem in parser.read_events():
                if(elem.tag == 'boardgame'):
                    bgg_id = elem.get('objectid')
        elif(result_count > 1):
            parser = ET.XMLPullParser(['start'])
            parser.feed(text)
            
            possible_games = dict()
            tmp_id = 0
            tmp_name = ""
            
            for event, elem in parser.read_events():
                if(elem.tag == 'boardgame'):
                    tmp_id = elem.get('objectid')
                if(elem.tag == 'name'):
                    tmp_name = elem.text
                    possible_games[tmp_name.lower()] = tmp_id
                     
            s = difflib.get_close_matches(name.lower(), list(possible_games.keys()), 1, 0.7)
            
    if(len(s) > 0):
        return possible_games.get(s[0])
    else:
        return -1

In [61]:
dataset = list()

with open('rules_dataset.csv') as csvfile:
    spamreader = csv.DictReader(csvfile)
    for row in spamreader:
        dataset.append(row)

updated_dataset = list()
      
for d in dataset:
    bbg_id = get_id_by_name(d.get('name'))
    # maintain the record only if it was possible to retrieve the id
    if(bbg_id != -1):
        weight, numweights = get_weight_by_id(bbg_id)
        d['numweights'] = int(numweights)
        d['weight'] = float(weight)
        d['id'] = bbg_id
        
        #Maintain the record only if the weight is not 0
        if d['weight'] != 0:
            updated_dataset.append(d)

12
38
5028
238
0
82
110
30
211
6230
52
46
82
1628
438
308
11
12
28
144
119
1273
0
4
20
456
0
53
116
1303
180
241
33
99
1
1
165
161
37
48
626
50
1399
857
0
1738
536
48
154
1043
218
138
207
1123
327
61
920
29
236
236
147
75
111
360
44
0
645
739
2
118
719
17
396
29
89
89
273
264
264
26
262
1004
0
127
56
0
0
30
203
0
0
0
24
0
613
0
0
0
0
0
0
0
142
0
0
0
28
0
0
253
0
0
53
0
301
0
0
0
753
0
0
147
0
0
0
238
1
525
54
0
0
0
1
0
0
0
1034
0
0
8
0
0
88
0
0
0
29
0
0
164
164
0
0
0
0
0
0
231
0
0
0
0
0
0
0
2
0
0
0
86
0
0
13
0
0
0
570
0
0
0
0
425
0
0
0
0
49
640
14
3
3
221
2274
3054
7
6
0
0
0
62
0
0
328
0
2325
2325
234
234
0
1
0
0
0
0
316
0
0
209
0
0
0
35
168
124
892
95
904
187
389
83
98
84
0
155
5832
36
2
0
34
41
1
25
7
329
619
26
101
153
261
12
600
80
0
0
32
31
59
251
52
1540
192
2059
7
2943
23
1987
0
1130
1
15
273
3389
32
38
25
18
805
236
787
0
27
222
895
1025
141
277
17
67
310
63
418
302
3608
1
147
0
17
18
111
55
10
1
113
0
1408
0
18
3780
0
4860
572
0
0
0
4
0
0
237
0
62
0
18
3
0
3
42
0
0
427
0
0
0
1

In [75]:
import pandas as pd

df = pd.DataFrame(updated_dataset)
df = df.drop(df.columns[0],axis = 1)
new_cols = ["id", "name", "year", "publisher", "weight", "numweights", "rules"]
df = df.drop_duplicates(subset=['name'], keep='first')
df = df.reindex(columns=new_cols)
df = df.loc[df['numweights']>50].reindex()
df

,id,name,year,publisher,weight,numweights,rules
2,68448,7 WONDERS,2010,ASMODEE,2.3214,5028,"setup depending on the number of players, retu..."
3,141517,A STUDY IN EMERALD,2013,TREEFROG GAMES,3.4538,238,setup shuffle the secret identity cards and de...
4,253499,A WAR OF WHISPERS,2020,STARLING GAMES,2.5488,82,setup each player chooses a matching player bo...
5,239472,ABOMINATION: HEIR OF FRANKENSTEIN,2019,PLAID HAT GAMES,3.2636,110,setup place the paris board in the center of t...
7,242705,AEON TRESPASS,2022,INTO THE UNKNOWN,4.7204,211,campaign setup find and prepare these elements...
...,...,...,...,...,...,...,...
215,22038,WARRIOR KNIGHTS,2006,FANTASY FLIGHT GAMES,3.5094,532,setup 1. each player (baron) picks a color and...
216,244654,WILDLANDS,2018,OSPREY GAMES,2.1250,88,setup choose a map to play on. each player cho...
218,38506,WITCH OF SALEM,2009,MAYFAIR GAMES,2.4832,149,"setup shuffle the 6 great old one (goo) cards,..."
222,8593,WRECKAGE,2003,FANTASY FLIGHT GAMES,1.7222,90,setup 1. starting with a randomly chosen first...


In [77]:
df.to_csv("bbg_boardgames_dataset_FILTERED.csv")